In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ShuffleSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error, make_scorer, root_mean_squared_error, mean_absolute_error
import xgboost as xgb

# Experimenting

In [139]:
df = pd.read_csv("car_data.csv")
df = df.drop("guid", axis=1)
X = df.copy()
X = pd.get_dummies(X, columns=["make", "model", "fuel_type", "transmission"])
y = X.pop("price")
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

In [140]:
scaler = StandardScaler()
X_train[["age", "mileage"]] = scaler.fit_transform(X_train[["age", "mileage"]])
X_test[["age", "mileage"]] = scaler.transform(X_test[["age", "mileage"]])

In [95]:
X

,mileage,age,make_abarth,make_alfa romeo,make_alpina,make_alpine,make_aston martin,make_audi,make_bentley,make_bmw,...,fuel_type_b,fuel_type_c,fuel_type_d,fuel_type_e,fuel_type_h,fuel_type_l,transmission_- Transmissie,transmission_Automatisch,transmission_Half/Semi-automaat,transmission_Handgeschakeld
0,252275,4374,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
1,96473,4678,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
2,248175,5775,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
3,8998,874,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
4,175478,12104,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9017,193000,6604,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
9018,179441,7242,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
9019,70187,3682,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
9020,119956,5263,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,False


## Random forrest

In [224]:
rfc = RandomForestRegressor(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestRegressor()

In [280]:
rf = RandomForestRegressor()
import numpy as np
param_grid = {
    'n_estimators': range(500,1500),  # Number of trees in the forest
    'max_depth': list(range(1, 100)) + [None],  # Maximum depth of the tree
    'min_samples_split': range(1,11),  # Minimum number of samples required to split a node
    'min_samples_leaf': range(1,5),    # Minimum number of samples required at each leaf node
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
    'bootstrap': [True, False]        # Whether bootstrap samples are used
    # 'max_samples': [None, 0.8, 0.9]    # If bootstrap=True, number of samples to draw
}
scoring = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
grid_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    scoring=scoring,
    n_iter = 100,
    cv=3,               # 5-fold cross-validation
    n_jobs=-1,          # Use all available cores
    verbose=3,          # Print progress
    refit=True          # Refit the best model on the entire dataset
)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\Nick\Desktop\ML_cars\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
147 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
66 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Nick\Desktop\ML_cars\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\Desktop\ML_cars\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\Nick\Desktop\ML_cars\Lib\site-packages\sklearn\base.py", line 436, in

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': range(1, 5),
                                        'min_samples_split': range(1, 11),
                                        'n_estimators': range(500, 1500)},
                   scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False, response_method='predict'),
                   verbose=3)

In [281]:
grid_search.best_params_

{'n_estimators': 1037,
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 97,
 'bootstrap': False}

In [282]:
yhat = grid_search.predict(X_test)

In [283]:
mean_absolute_percentage_error(y_test, yhat)

0.3457807085109395

In [289]:
y_test.head()

4207     64950
8273     11750
6189     12445
5923     17445
627     149950
Name: price, dtype: int64

In [288]:
yhat[:5]

array([71764.86176788, 11066.08733077, 11397.23118534, 18222.71658781,
       98008.03230473])

## XGB

In [150]:
df = pd.read_csv("car_data.csv")
df = df.drop("guid", axis=1)
X = df.copy()
y = X.pop("price")
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

In [151]:
scaler = StandardScaler()
# X_train[["age", "mileage"]] = scaler.fit_transform(X_train[["age", "mileage"]])
# X_test[["age", "mileage"]] = scaler.transform(X_test[["age", "mileage"]])
X_numeric = X[["age", "mileage"]]
X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)

C:\Users\Nick\AppData\Local\Temp\ipykernel_4664\333700207.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)


In [154]:
enc = OneHotEncoder(sparse_output=False)
X_cat = X[["make", "model", "fuel_type", "transmission"]]
X_cat = pd.DataFrame(enc.fit_transform(X_cat)).astype(bool)

In [ ]:
X_combined = X_numeric.join(X_cat)
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, shuffle=True, random_state=1)

TypeError: got an unexpected keyword argument 'seed'

In [141]:
import xgboost as xgb
reg = xgb.XGBRegressor(n_estimators=5000, max_depth=7, eta=0.1)
reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, feature_weights=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=5000, n_jobs=None, ...)

In [142]:
yhat2 = reg.predict(X_test)

In [145]:
from sklearn.metrics import mean_absolute_error
mean_absolute_percentage_error(y_test, yhat2)

0.27939990162849426

In [158]:
# averaging with a loop
results = []
for _ in range(10):
    df = pd.read_csv("car_data.csv")
    df = df.drop("guid", axis=1)
    X = df.copy()
    X = pd.get_dummies(X, columns=["make", "model", "fuel_type", "transmission"])
    y = X.pop("price")
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
    scaler = StandardScaler()
    X_train[["age", "mileage"]] = scaler.fit_transform(X_train[["age", "mileage"]])
    X_test[["age", "mileage"]] = scaler.transform(X_test[["age", "mileage"]])
    reg = xgb.XGBRegressor(n_estimators=5000, max_depth=7, eta=0.1)
    reg.fit(X_train, y_train)
    yhat2 = reg.predict(X_test)
    results.append(root_mean_squared_error(y_test,yhat2))
print(np.mean(results))

18225.65927734375


In [159]:
# averaging with a loop
results = []
for _ in range(10):
    df = pd.read_csv("car_data.csv")
    df = df.drop("guid", axis=1)
    X = df.copy()
    y = X.pop("price")
    scaler = StandardScaler()
    X_numeric = X[["age", "mileage"]]
    X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)
    enc = OneHotEncoder(sparse_output=False)
    X_cat = X[["make", "model", "fuel_type", "transmission"]]
    X_cat = pd.DataFrame(enc.fit_transform(X_cat)).astype(bool)
    X_combined = X_numeric.join(X_cat)
    X_train, X_test, y_train, y_test = train_test_split(X_combined, y, shuffle=True, random_state=1)
    reg = xgb.XGBRegressor(n_estimators=5000, max_depth=7, eta=0.1)
    reg.fit(X_train, y_train)
    yhat2 = reg.predict(X_test)
    results.append(root_mean_squared_error(y_test,yhat2))
print(np.mean(results))

C:\Users\Nick\AppData\Local\Temp\ipykernel_4664\3620298072.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)
C:\Users\Nick\AppData\Local\Temp\ipykernel_4664\3620298072.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)
C:\Users\Nick\AppData\Local\Temp\ipykernel_4664\3620298072.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

12884.8662109375


## GBR

In [166]:
xnum = X[["age", "mileage"]].head()
xcat = X[["make", "model", "fuel_type", "transmission"]].head()
xcat = pd.DataFrame(enc.transform(xcat)).astype(bool)
xnum[["age", "mileage"]] = scaler.transform(xnum)
xcomb = xnum.join(xcat)
reg.predict(xcomb)

array([ 2678.5576,  6005.7197,  3147.0212, 16688.838 ,   909.983 ],
      dtype=float32)

In [170]:
y.head()

0     2495
1     4950
2     2888
3    16495
4      490
Name: price, dtype: int64

In [113]:
from sklearn.ensemble import GradientBoostingRegressor
reg2 = GradientBoostingRegressor(n_estimators=500, max_depth=7, learning_rate=0.1)
reg2.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=7, n_estimators=500)

In [ ]:
yhat2 = reg2.predict(X_test)
mean_absolute_percentage_error(y_test, yhat2)

3.7626030445098877

# Pipeline making

In [ ]:
# averaging with a loop
results = []
for _ in range(10):
    df = pd.read_csv("car_data.csv")
    df = df.drop("guid", axis=1)
    X = df.copy()
    y = X.pop("price")
    scaler = StandardScaler()
    X_numeric = X[["age", "mileage"]]
    X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)
    enc = OneHotEncoder(sparse_output=False)
    X_cat = X[["make", "model", "fuel_type", "transmission"]]
    X_cat = pd.DataFrame(enc.fit_transform(X_cat)).astype(bool)
    X_combined = X_numeric.join(X_cat)
    X_train, X_test, y_train, y_test = train_test_split(X_combined, y, shuffle=True, random_state=1)
    reg = xgb.XGBRegressor(n_estimators=5000, max_depth=7, eta=0.1)
    reg.fit(X_train, y_train)
    yhat2 = reg.predict(X_test)
    results.append(root_mean_squared_error(y_test,yhat2))
print(np.mean(results))

In [78]:
def load_data():
    df = pd.read_csv("car_data.csv")
    df = df.drop("guid", axis=1)
    X = df.copy()
    y = X.pop("price")
    return X,y

def scale_encode_training(X,y, y_scaler:StandardScaler):
    scaler = StandardScaler()
    enc = OneHotEncoder(sparse_output=False)

    y = y_scaler.fit_transform(pd.DataFrame(y))

    X_numeric = X[["age", "mileage"]]
    X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)

    X_cat = X[["make", "model", "fuel_type", "transmission"]]
    X_cat = pd.DataFrame(enc.fit_transform(X_cat)).astype(bool)
    X_combined = X_numeric.join(X_cat)

    return X_combined, y, y_scaler

scaler = StandardScaler()
X, y = load_data()
X,y,scaler = scale_encode_training(X,y,scaler)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

C:\Users\Nick\AppData\Local\Temp\ipykernel_5984\3493880333.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_numeric[["age", "mileage"]] = scaler.fit_transform(X_numeric)


In [40]:
reg = xgb.XGBRegressor()
param_grid = {
    'n_estimators': np.arange(1000,10000, 2000).tolist(),
    'learning_rate': np.arange(0.001, 0.1, 0.01).tolist(),
    'max_depth': np.arange(3,9).tolist(),
    'min_child_weight': np.arange(1,6).tolist(),
    'gamma': np.arange(0, 0.1, 0.01).tolist(),
    'alpha': np.arange(0, 0.1, 0.01).tolist(),
    'lambda': [1, 1.5]
}


# Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=2, scoring='neg_mean_squared_error', n_jobs=4, verbose=2)
grid_search = RandomizedSearchCV(estimator=reg, param_distributions=param_grid, cv=3, n_iter=100, scoring='neg_mean_squared_error', n_jobs=4, verbose=2, refit=True)

In [41]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          feature_weights=None, gamma=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraint...
                                                  0.05, 0.06, 0.07, 0.08,
                                                  0.09],
                                        'lambda': [1, 1.5],
                                        'learning_rate': [0.001, 0.011,
                                                          0.020999999999999998,
                                                          0.030999999999999996,
                                                          0.040999999999999995,
                                                          0.05099999999999999,
                                                          0.06099999999999999,
                                                          0.071,
                                                          0.08099999999999999,
                                                          0.09099999999999998],
                                        'max_depth': [3, 4, 5, 6, 7, 8],
                                        'min_child_weight': [1, 2, 3, 4, 5],
                                        'n_estimators': [1000, 3000, 5000, 7000,
                                                         9000]},
                   scoring='neg_mean_squared_error', verbose=2)

In [42]:
yhat = grid_search.best_estimator_.predict(X_test)
mean_absolute_percentage_error(y_true=y_test, y_pred=yhat)

0.4331547021865845

In [43]:
grid_search.best_params_

{'n_estimators': 1000,
 'min_child_weight': 1,
 'max_depth': 5,
 'learning_rate': 0.030999999999999996,
 'lambda': 1,
 'gamma': 0.04,
 'alpha': 0.02}

In [ ]:
reg = xgb.XGBRegressor(n_estimators=5000, max_depth=5, eta=0.03, min_child_weight=1, reg_lambda=1, reg_alpha=0.2, gamma=0.04)
reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=5000,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
yhat = reg.predict(X_test)

19671.189067445408

In [86]:
yhat = scaler.inverse_transform(pd.DataFrame(yhat))
# y_test = scaler.inverse_transform(pd.DataFrame(y_test))

In [ ]:
mean_absolute_percentage_error(y_true=y_test, y_pred=yhat)

array([[11499.],
       [18900.],
       [ 6745.],
       ...,
       [33699.],
       [17950.],
       [17450.]], shape=(2256, 1))

In [97]:
results_MEA = []
results_MEPA = []
for _ in range(10):
    reg = xgb.XGBRegressor(n_estimators=5000, max_depth=5, eta=0.03, min_child_weight=1, reg_lambda=1, reg_alpha=0.2, gamma=0.04)
    reg.fit(X_train, y_train)
    yhat = reg.predict(X_test)
    yhat = scaler.inverse_transform(pd.DataFrame(yhat))
    results_MEA.append(mean_absolute_error(y_true=y_test, y_pred=yhat))
    results_MEPA.append(mean_absolute_percentage_error(y_true=y_test, y_pred=yhat))

In [98]:
print(np.mean(results_MEA))
print(np.mean(results_MEPA))

4449.927329354252
0.32028752069290184
